In [10]:
import pandas as pd
import numpy as np

from statsmodels.tsa.arima.model import ARIMA


In [11]:
df = pd.read_csv(
    "../atp_transformed/2000-2024 players_3.csv",
    parse_dates=["tourney_date"]
)

df = df.sort_values(["player_id", "tourney_date"])

# Convert tourney_date to datetime
df['tourney_date_datetime'] = pd.to_datetime(df['tourney_date'])

# Extract year
df['year'] = df['tourney_date_datetime'].dt.year

df = df[(df["year"] >= 2013) & (df["year"] <= 2023)]


C:\Users\jesse\AppData\Local\Temp\ipykernel_31452\3229348571.py:1: DtypeWarning: Columns (18,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [12]:
pid = 104745  # example: Rafael Nadal

p_df = df[df.player_id == pid].sort_values("year")

# 3-year rolling average of player_rank
p_df["rank_3yr_avg"] = p_df["player_rank"].rolling(3, min_periods=3).mean()

print(p_df[["year", "player_rank", "rank_3yr_avg"]])

        year  player_rank  rank_3yr_avg
463325  2013          5.0           NaN
463389  2013          2.0           NaN
463388  2013          2.0      3.000000
463387  2013          2.0      2.000000
463386  2013          2.0      2.000000
...      ...          ...           ...
463867  2022          5.0      4.333333
463906  2023          2.0      3.666667
463904  2023          2.0      3.000000
463905  2023          2.0      2.000000
463907  2023          2.0      2.000000

[583 rows x 3 columns]


In [13]:
MIN_YEARS = 3  # minimum 3 years for rolling average

if len(p_df) < MIN_YEARS:
    raise ValueError("Not enough data to forecast")


In [26]:
### TBD : meerdere features

X = pd.get_dummies(
    X,
    columns=["tourney_level"],
    drop_first=True
)

y = p_df["rank_3yr_avg"].dropna()
X = p_df.loc[y.index, ['tourney_level', 'match_num', 'player_age', 'days_of_experience' ]]
                       #'draw_size_rolling_med_10', 
                       #'highest_finish_position_rolling_med_10', 
                       #'elo_pre_match_rolling_mean_10',
                       #'opponent_elo_pre_match_rolling_mean_10', 
                       #'set_dominance_rolling_mean_10', 
                       #'tb_dominance_rolling_mean_10', 
                       #'mean_numb_rolling_mean_10',  
                       #'mean_diff_rolling_mean_10',  
                       #'mean_tb_numb_rolling_mean_10', 
                       #'mean_tb_diff_rolling_mean_10']]

model = ARIMA(y, exog=X, order=(0,1,0), trend="t")
res = model.fit()
print(res.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [29]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

y = p_df["rank_3yr_avg"].dropna()

X = p_df.loc[y.index, [
    "tourney_level",
    "match_num",
    "player_age",
    "days_of_experience"
]]

#print(X.dtypes)

X = pd.get_dummies(
    X,
    columns=["tourney_level"],
    drop_first=True
)

X = X.apply(lambda s: pd.to_numeric(s, errors="coerce"))

print(X.dtypes)

data = pd.concat([y, X], axis=1).dropna()

y_clean = data["rank_3yr_avg"]
X_clean = data.drop(columns="rank_3yr_avg")

non_numeric = X_clean.select_dtypes(include=["object", "category"])
assert non_numeric.empty

assert np.isfinite(X_clean.to_numpy()).all()
assert np.isfinite(y_clean.to_numpy()).all()

model = ARIMA(
    y_clean,
    exog=X_clean,
    order=(0, 1, 0),
    trend="t"
)

res = model.fit()
print(res.summary())


match_num               int64
player_age            float64
days_of_experience      int64
tourney_level_D          bool
tourney_level_F          bool
tourney_level_G          bool
tourney_level_M          bool
dtype: object


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

y = p_df["rank_3yr_avg"].dropna()

X = p_df.loc[y.index, [
    "tourney_level",
    "match_num",
    "player_age",
    "days_of_experience"
]]

X = pd.get_dummies(
    X,
    columns=["tourney_level"],
    drop_first=True
)

X = X.apply(pd.to_numeric, errors="coerce")

data = pd.concat([y, X], axis=1).dropna()

y_clean = data["rank_3yr_avg"]
X_clean = data.drop(columns="rank_3yr_avg").astype(float)

model = ARIMA(
    y_clean,
    exog=X_clean,
    order=(0, 1, 0),
    trend="t"
)

res = model.fit()
print(res.summary())


In [15]:
# Target: 3-year rolling rank average
#y = p_df["rank_3yr_avg"].dropna()

#model = ARIMA(
#    y,
#    order=(0,1,0),  # random walk with drift
#    trend="t"
#)

#res = model.fit()
#print(res.summary())


In [19]:
last_X = X.iloc[-1]

X_future = pd.DataFrame(
    {
        "player_age": [last_X["player_age"] + 1],       
        "days_of_experience": [last_X["days_of_experience"] + 365]
    },
    index=[X.index[-1] + 1]
)

forecast = res.forecast(steps=1, exog=X_future)

c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [20]:
forecast = res.forecast(steps=1, exog=X_future)
rank_1y = forecast.iloc[-1]
print("Predicted player rank in 1 year:", rank_1y)


Predicted player rank in 1 year: 1.8031297051523438


c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [21]:
def rank_to_bin(rank):
    if rank <= 10:
        return 1
    elif rank <= 25:
        return 2
    elif rank <= 50:
        return 3
    elif rank <= 100:
        return 4
    elif rank <= 250:
        return 5
    else:
        return 6

predicted_bin = rank_to_bin(rank_1y)
print("Predicted rank bin:", predicted_bin)


Predicted rank bin: 1


In [23]:
forecast_obj = res.get_forecast(steps=1, exog=X_future)
conf_int = forecast_obj.conf_int()
conf_int_1y = conf_int.iloc[-1]

print("95% confidence interval for player rank in 1 year:")
print(conf_int_1y)


95% confidence interval for player rank in 1 year:
lower rank_3yr_avg    0.940540
upper rank_3yr_avg    2.665719
Name: 581, dtype: float64


c:\Users\jesse\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
